# Fetch (and process) forecasts from FluSight repository
Used to download files and develop functions to manipulate them

In [20]:
from colorama import Style, Fore
import numpy as np
import pandas as pd
import os
import sys
from pathlib import Path
import time

os.chdir("..")

import rtrend_interface.flusight_tools as fst
import rtrend_interface.truth_data_structs as tds
from rtrend_forecast.postprocessing import aggregate_in_periods
from toolbox.file_tools import download_file_simple

# MANUALLY ADD RTREND LIBRARY PATH
sys.path.append("../../../Rtrend_project/")
from rtrend_forecast.scoring import weighted_interval_score_fast

# ---------
q_edges = np.array([
    0.01, 0.025, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35,
    0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7 , 0.75, 0.8, 0.85, 0.9,
    0.95, 0.975, 0.99])
q_alphas = 2 * q_edges[: (q_edges.shape[0] + 1) // 2]

In [2]:
# =========== Some quick (maybe temporary) definitions
def aggregate_to_weekly_truth(daily_sr: pd.Series, ref_tlabel=pd.Timestamp("2023-10-08")):
    vals = list()
    keys = daily_sr.index.get_level_values("location").unique()
    for loc_id in keys:
        # Selects data from that location and aggregates
        vals.append(
            aggregate_in_periods(
                daily_sr.xs(loc_id, level="location"),
                pd.Timedelta("1w"),
                ref_tlabel,  # Ref label for aggregation
                full_only=True,
        ))
        
    return pd.concat(vals, keys=keys).reorder_levels([1, 0])

# Fetch data from FluSight github

In [3]:
do_download = False
url_fmt = "https://raw.githubusercontent.com/paulocv/FluSight-forecast-hub/main/model-output/FluSight-baseline/{}-FluSight-baseline.csv"
out_dir = Path("forecast_out/FluSight-Baseline")

ref_dates = pd.date_range("2023-10-14", "2024-05-04", freq="w-SAT")

# --------------

# Loop over each date to download the file for
for i_date, date in enumerate(ref_dates):
    url = url_fmt.format(date.date().isoformat())
    fname = out_dir.joinpath(os.path.basename(url))
    
    print(f"Downloading: {url} into {fname}")
    if do_download:
        try:
            download_file_simple(url, fname, makedirs=True)
        except RuntimeError as e:
            print(f"{Fore.RED}ERROR. {e}{Style.RESET_ALL}")
            
        print("Sleeping a bit")
        time.sleep(5)
    else: 
        print("SKIPPED (do_download = False)")
        
print("===============================\nDONE")
    # download_file_simple()

Downloading: https://raw.githubusercontent.com/paulocv/FluSight-forecast-hub/main/model-output/FluSight-baseline/2023-10-14-FluSight-baseline.csv into forecast_out/FluSight-Baseline/2023-10-14-FluSight-baseline.csv
SKIPPED (do_download = False)
Downloading: https://raw.githubusercontent.com/paulocv/FluSight-forecast-hub/main/model-output/FluSight-baseline/2023-10-21-FluSight-baseline.csv into forecast_out/FluSight-Baseline/2023-10-21-FluSight-baseline.csv
SKIPPED (do_download = False)
Downloading: https://raw.githubusercontent.com/paulocv/FluSight-forecast-hub/main/model-output/FluSight-baseline/2023-10-28-FluSight-baseline.csv into forecast_out/FluSight-Baseline/2023-10-28-FluSight-baseline.csv
SKIPPED (do_download = False)
Downloading: https://raw.githubusercontent.com/paulocv/FluSight-forecast-hub/main/model-output/FluSight-baseline/2023-11-04-FluSight-baseline.csv into forecast_out/FluSight-Baseline/2023-11-04-FluSight-baseline.csv
SKIPPED (do_download = False)
Downloading: https:/

# Import data to send into the training pipeline (as a baseline)

In [4]:
example_baseline_fname = Path("forecast_out/FluSight-Baseline/2023-11-18-FluSight-baseline.csv")
example_truth_fname = Path("hosp_data/past_BKP/truth_weekly_latest_2024-01-02.csv")

# ------------

fore_obj = fst.FluSight2023Fore(example_baseline_fname)
truth_sr = tds.load_truth_cdc_simple(example_truth_fname)["value"]

In [13]:
# Prepare data for scoring
weekly_truth_df = aggregate_to_weekly_truth(truth_sr)
# fore_obj.quantiles_df  # Signature

# --- Example. California 
loc_fips = "06"
fore_df = fore_obj.quantiles_df.xs(loc_fips, level="location")
state_truth_sr = truth_sr.xs(loc_fips, level="location")
# truth_sr = truth_df.xs(loc_fips, level="location")["value"]

# Align truth with forecast dates
if not fore_df.index.isin(state_truth_sr.index).all():
    raise ValueError("Dates in the forecast period don't all match truth. Please inspect.")
state_truth_sr = state_truth_sr.loc[fore_df.index]



target_end_date
2023-11-11     50.0
2023-11-18     65.0
2023-11-25     85.0
2023-12-02    107.0
2023-12-09    103.0
Name: value, dtype: float64

In [22]:
wis, wis_sharp, wis_calib = weighted_interval_score_fast(
    state_truth_sr.to_numpy(),
    q_alphas,
    fore_df,
)

wis

array([762.65709156, 502.79862657, 403.72762817, 332.48195312,
       341.33809246])